In [1]:
import numpy as np
from header import *
from nsp.utils.base_conv import *

In [2]:
N = 4
X = np.random.randn(N**2, N**2)
X = X.T + X
# mask = np.abs(X)>=(np.max(X)/(N))
# X[mask] = 0
loss_l1 = nsp.loss.MES(torch.from_numpy(X), [N, N])
t = 0.001
ret_min_grad = 1e10
model = nsp.model.UnitaryRiemanGenerator(N, dtype=torch.float64)


In [3]:
X1 = np.random.randn(N, N)
X2 = np.random.randn(N, N)
X3 = np.random.randn(N, N)

Y = np.kron(X1, X2)
Y = np.kron(Y, X3)

In [4]:
Y.reshape(N,N,N,N,N,N, order='F').reshape(N**3, N**3)[N**2 + N, 0], Y[N+1, 0], X1[0,0] * X2[1,0] * X3[1,0]

(0.0071850677481504165, 0.0071850677481504165, 0.0071850677481504165)

In [5]:
# loss_l1._transform_einsum([model.matrix(), model2.matrix()])

In [6]:
M = loss_l1.X.reshape(loss_l1.act.tolist()*2)
M = torch.zeros_like(M)

In [7]:
I_not = torch.logical_not(torch.eye(4))

In [8]:
act = loss_l1.act.tolist()
I_not1 = torch.logical_not(torch.eye(act[0]))
I_not2 = torch.logical_not(torch.eye(act[1]))
ML = torch.kron(I_not1, torch.eye(act[1]))
MR = torch.kron(torch.eye(act[0]), I_not2)
MI = torch.kron(I_not1, I_not2) + torch.eye(np.prod(act))

In [9]:
X = torch.from_numpy(X)
XI = X * MI
XL = X * ML
XR = X * MR


In [10]:
X_3site = (torch.kron(torch.eye(N),XL) + torch.kron(XR,torch.eye(N)))
X_2 = XI

In [19]:
L = 4
bonds = [[i, (i+1)%L] for i in range(L)]
LH = sum_ham(X, bonds, L, 4)
LH_abs = sum_ham(stoquastic(X), bonds, L, 4)

In [20]:
np.linalg.eigvalsh(LH_abs)[-1], np.linalg.eigvalsh(LH)[-1]

(66.69024444031078, 27.035349234148846)

### added first then stoquastic

In [14]:
L = 4
bonds = [[i, (i+1)%L] for i in range(L)]
LH = sum_ham(X_2, bonds, L, 4)
bonds = [[i, (i+1)%L, (i+2)%L] for i in range(L)]
LH += sum_ham(X_3site,bonds, L, 4)

In [18]:
np.linalg.eigvalsh(LH_abs)[-1], np.linalg.eigvalsh(LH)[-1]

(60.90619596800536, 27.035349234148846)

In [16]:
L = 4
bonds = [[i, (i+1)%L] for i in range(L)]
LH_abs = sum_ham(stoquastic(X_2), bonds, L, 4)
bonds = [[i, (i+1)%L, (i+2)%L] for i in range(L)]
LH_abs += sum_ham(stoquastic(X_3site),bonds, L, 4)

In [17]:
np.linalg.eigvalsh(LH_abs)[-1], np.linalg.eigvalsh(LH)[-1]

(60.90619596800536, 27.035349234148846)

## apply on MG

In [125]:
import numpy as np
import argparse
import sys
sys.path.append('../nsp')
from nsp.utils.func import *


Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


SzSz = np.kron(Sz,Sz).real.astype(np.float64)
SxSx = np.kron(Sx,Sx).real.astype(np.float64)
SySy = np.kron(Sy,Sy).real.astype(np.float64)

lh = SzSz + SxSx + SySy

In [126]:
print("1D chain lattice")
assert L%3 == 0, "L should be 3*K "
bonds = [[0,1], [0, 2], [1, 2]]
lh2 = sum_ham(lh/2, bonds, 3, 2)
LH = sum_ham(lh2/2, [[0,1,2], [3, 4, 5]], 6, 2) + sum_ham(lh2, [[1, 2, 3], [2, 3, 4]], 6, 2)
LH = -LH

1D chain lattice


### original

In [145]:
L = 12
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H = sum_ham(LH, bonds, int(L/3), 8)
print("max_eig : ", np.linalg.eigvalsh(H)[-1])

max_eig :  4.499999999999985


In [146]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_abs = sum_ham(stoquastic(LH), bonds, int(L/3), 8)
print("max_eig : ", np.linalg.eigvalsh(H_abs)[-1])

max_eig :  6.411629916792666


## optim

In [129]:
X = LH # for N = 8
N = 8
loss_mes = nsp.loss.MES(X, [N, N])
best_fun = 1E10
for _ in range(1):
    model = nsp.model.UnitaryRiemanGenerator(N, dtype=torch.float64)
    solver = UnitaryTransTs(RiemanUnitaryCG, model, loss_mes, lr = 0.005, momentum=0.1)
    ret = solver.run(10000, False)
    if ret.fun < best_fun:
        print(f"\nbest_fun : {ret.fun}\n")
        best_fun = ret.fun
        best_model = model

LH_o = loss_mes._transform([best_model.matrix()]*loss_mes._n_unitaries, original = True).detach().numpy()

target loss      : 2.2500000000
initial loss     : 5.2466106372
dtype changes from torch.float64 to torch.float32
loss upper bound : 2.8182671070




  0%|          | 0/10000 [00:00<?, ?it/s]

dtype changes from torch.float32 to torch.float64
No local minimum found
7.563405611135426e-13 1.0887419921346009e-14 2.740214187062595 2.7402141870625947
No local minimum found
3.203911359522078e-12 2.881776308640838e-13 2.740212774506367 2.7402127745063645
No local minimum found
3.4966026685994115e-12 1.8899414688348762e-12 2.740212774506367 2.740212774506363
stack in local minimum --> break loop

best_fun : 2.7402127745063694



In [147]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_o = sum_ham(LH_o, bonds, int(L/3), 8)
print("max eig",np.linalg.eigvalsh(H_o)[-1])

max eig 4.499999999998592


In [148]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_o_abs = sum_ham(stoquastic(LH_o), bonds, int(L/3), 8)
print("max eig",np.linalg.eigvalsh(H_o_abs)[-1])

max eig 6.05378867487443


### add_first

In [149]:
act = loss_mes.act.tolist()
I_not1 = torch.logical_not(torch.eye(act[0]))
I_not2 = torch.logical_not(torch.eye(act[1]))
ML = torch.kron(I_not1, torch.eye(act[1]))
MR = torch.kron(torch.eye(act[0]), I_not2)
MI = torch.kron(I_not1, I_not2) + torch.eye(np.prod(act))

In [150]:
LH_3_site = (torch.kron(torch.eye(N),ML * LH_o) + torch.kron(MR * LH_o,torch.eye(N)))
LH_2_site = MI * LH_o

In [151]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_af = sum_ham(LH_2_site, bonds, int(L/3), 8)
bonds = [[i, (i+1)%int(L/3), (i+2)%int(L/3)] for i in range(int(L/3))]
H_af += sum_ham(LH_3_site, bonds, int(L/3), 8)

In [152]:
print("max eig of add first",np.linalg.eigvalsh(H_af)[-1])

max eig of add first 4.499999999998592


In [153]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_af_abs = sum_ham(stoquastic(LH_2_site), bonds, int(L/3), 8)
bonds = [[i, (i+1)%int(L/3), (i+2)%int(L/3)] for i in range(int(L/3))]
H_af_abs += sum_ham(stoquastic(LH_3_site), bonds, int(L/3), 8)

In [154]:
print("max eig of add first (abs)",np.linalg.eigvalsh(H_af_abs)[-1])

max eig of add first (abs) 5.90262796484217
